In [1]:

%load_ext sql

import pandas as pd
import sqlite3
%sql sqlite:///Course_DW.db 

'Connected: @Course_DW.db'

In [2]:
%%sql
ATTACH DATABASE 'CourseData.db' AS cd

 * sqlite:///Course_DW.db
Done.


[]

In [4]:
%%sql
SELECT *
FROM cd.PROGRAM
LIMIT 5

 * sqlite:///Course_DW.db
(sqlite3.OperationalError) no such table: cd.PROGRAMS
[SQL: SELECT *
FROM cd.PROGRAMS
LIMIT 5]
(Background on this error at: http://sqlalche.me/e/e3q8)


In [ ]:
%%sql

DROP TABLE IF EXISTS LOCATION1;
CREATE TABLE LOCATION1 (
    LocationID INTEGER PRIMARY KEY,
    Location_Code TEXT,
    Building TEXT,
    Room TEXT
);

DROP TABLE IF EXISTS INSTRUCTOR1;
CREATE TABLE INSTRUCTOR1 (
    InstructorID INTEGER PRIMARY KEY,
    InstructorName TEXT 
);

DROP TABLE IF EXISTS PROGRAM1;
CREATE TABLE PROGRAM1 (
    ProgramID INTEGER PRIMARY KEY,
    ProgramCode TEXT,
    ProgramName TEXT
);

DROP TABLE IF EXISTS COURSE1;
CREATE TABLE COURSE1 (
    CatalogID TEXT,
    CourseID INTEGER PRIMARY KEY,
    ProgramID INTEGER NOT NULL,
    CourseTitle TEXT NOT NULL,
    Description TEXT,
    Attributes TEXT,
    Coreqs TEXT,
    Prereqs TEXT,
    Fees TEXT,
    Credits INTEGER NOT NULL,
    FOREIGN KEY (ProgramID) REFERENCES PROGRAM (ProgramID)
);

DROP TABLE IF EXISTS COURSE_MEETING1;
CREATE TABLE COURSE_MEETING1 (
    CourseMeetingID INTEGER PRIMARY KEY,
    CatalogID TEXT,
    Location TEXT,
    StartDateTime TEXT,
    EndDateTime TEXT,
    FOREIGN KEY (Location) REFERENCES LOCATION (Location)
);

In [ ]:
%%sql
DELETE FROM LOCATION1;
INSERT INTO LOCATION1 (Location_Code, Building, Room)
SELECT Location_Code, Building, Room
FROM cd.LOCATION;

In [ ]:
%%sql
DROP TABLE IF EXISTS PROGRAM1;
CREATE TABLE PROGRAM1 (
    ProgramID INTEGER PRIMARY KEY,
    ProgramCode TEXT,
    ProgramName TEXT
);

In [ ]:
%%sql

DROP TABLE IF EXISTS COURSE1;
CREATE TABLE COURSE1 (
    CatalogID TEXT,
    CourseID INTEGER PRIMARY KEY,
    ProgramID INTEGER NOT NULL,
    CourseTitle TEXT NOT NULL,
    Description TEXT,
    Attributes TEXT,
    Coreqs TEXT,
    Prereqs TEXT,
    Fees TEXT,
    Credits INTEGER NOT NULL,
    FOREIGN KEY (ProgramID) REFERENCES PROGRAM (ProgramID)
);

In [ ]:
%%sql

DROP TABLE IF EXISTS COURSE_MEETING;
CREATE TABLE COURSE_MEETING (
    CourseMeetingID INTEGER PRIMARY KEY,
    CatalogID TEXT,
    Location TEXT,
    StartDateTime TEXT,
    EndDateTime TEXT,
    FOREIGN KEY (Location) REFERENCES LOCATION (Location)
);

In [ ]:
-- Sample code
%%sql
DELETE FROM INSTRUCTOR1;
INSERT INTO INSTRUCTOR1(Instructor_id, Name, Phone, Email ) 
    SELECT Instructor_id, Name, Phone, Email 
    FROM coursedata.INSTRUCTOR;